In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
from sklearn.metrics import fbeta_score
import numpy as np

In [3]:
import sys,os
sys.path.append('../')
import ama

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


IndentationError: unexpected indent (submissions.py, line 98)

In [6]:
import csv
import numpy as np

def read_val_data(fp):
    with open(fp) as csvf:
        reader = csv.reader(csvf)
        headers = reader.next()
        guesses = []
        trues = []
        filenames = []
        for row in reader:
            guesses.append(np.array(row[1:18], dtype=float))
            trues.append(np.array(row[18:], dtype=float))
            filenames.append(row[0])
    guesses = np.stack(guesses)
    trues = np.stack(trues)
    return filenames, trues, guesses

def tag_guesses(guesses, thresholds):
    return np.array(guesses>thresholds, dtype=float)
                

def f2(y_true, y_pred):
    beta = 2
    
    tp = np.sum(y_true*y_pred)
    fp = np.sum(np.clip(y_pred - y_true, 0, 1))
    fn = np.sum(np.clip(y_true - y_pred, 0, 1))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall)

def find_f2_thresholds(trues, guesses):
    
    best = 0
    best_score = -1
    totry = np.arange(0,1,0.005)
    for v in totry:
        thresholds = np.array([v for _ in range(17)])
        score = f2(trues, tag_guesses(guesses, thresholds))
        if score>best_score:
            best_score = score
            best = v
    thresholds = np.array([best for _ in range(17)])
        
    return thresholds

In [8]:
fp = '../submissions/xgb_train.csv'

In [5]:
filenames, trues, guesses = read_val_data(fp)

NameError: name 'read_val_data' is not defined

In [10]:
thresholds = find_f2_thresholds(trues, guesses)

In [11]:
f2(trues, tag_guesses(guesses, thresholds))

0.94417695234149634

In [14]:
f2(trues, tag_guesses(guesses, np.array([0.16 for _ in range(17)])))

0.93812664191491069

In [17]:
thresholds

array([ 0.245,  0.245,  0.245,  0.245,  0.245,  0.245,  0.245,  0.245,
        0.245,  0.245,  0.245,  0.245,  0.245,  0.245,  0.245,  0.245,
        0.245])

In [18]:
preds = tag_guesses(guesses, thresholds)

In [19]:
def read_presubmission(fp):
    with open(fp,'r') as csvf:
        reader = csv.reader(csvf)
        headers = reader.next()
        filenames = []
        guesses = []
        for row in reader:
            filenames.append(row[0])
            guesses.append(np.array(row[1:], dtype=float))
    return filenames, np.stack(guesses)

In [20]:
fp = '../submissions/xgb_presub.csv'
write_fp = '../submissions/xgb_thresh_val_set_all_together.csv'

In [21]:
filenames, guesses = read_presubmission(fp)

In [22]:
labelorder =  ["agriculture",
            "artisinal_mine",
            "bare_ground",
            "blooming",
            "blow_down",
            "clear",
            "cloudy",
            "conventional_mine",
            "cultivation",
            "habitation",
            "haze",
            "partly_cloudy",
            "primary",
            "road",
            "selective_logging",
            "slash_burn",
            "water"
        ]

In [23]:
labelorder_dan = ['slash_burn',
    'cultivation',
    'habitation',
    'blooming',
    'water',
    'blow_down',
    'bare_ground',
    'selective_logging',
    'cloudy',
    'agriculture',
    'artisinal_mine',
    'primary',
    'road',
    'clear',
    'partly_cloudy',
    'haze',
    'conventional_mine'
]

In [24]:
len(labelorder_dan)

17

In [25]:
ama.submissions.write_submission(guesses, filenames, labelorder, thresholds, write_fp)

In [70]:
from IPython.display import FileLink

In [71]:
FileLink(write_fp)

/home/ubuntu/persist/amazon/submissions/atc_1_test_sub_thresh_val_set_all_together.csv